In [1]:
# %load_ext autoreload
# %autoreload 2


In [2]:
import os
import torch


In [3]:
# random seed for reproducibility
torch.manual_seed(42)


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
print(torch.cuda.device_count())


cuda
1


In [5]:
# !pip -q install sentencepiece
# !pip -q install datasets
# !pip -q install fairscale
# !pip -q install transformers
# !pip -q install tqdm


In [6]:

# CRITICAL: Import the correct model.
# from models.model_baseline_nokv_reduced import ModelArgs, Transformer
from models.model_baseline_nokv import ModelArgs, Transformer
from tqdm import tqdm
from transformers import LlamaTokenizer

tokenizer = LlamaTokenizer("./tokenizers/tokenizer.model")
tokenizer.pad_token = tokenizer.eos_token

model_args = ModelArgs()
model_args.vocab_size = len(tokenizer)


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
# !export RANK=0


In [8]:
from fairscale.nn.model_parallel import initialize_model_parallel
import torch.distributed as dist

def setup(rank, world_size):
    "Sets up the process group and configuration for PyTorch Distributed Data Parallelism"
    os.environ["MASTER_ADDR"] = 'localhost'
    os.environ["MASTER_PORT"] = "12355"

    # Initialize the process group
    if not dist.is_initialized():
        dist.init_process_group("gloo", rank=rank, world_size=world_size)

def cleanup():
    "Cleans up the distributed environment"
    dist.destroy_process_group()

setup(0, 1)


# COMMENT IF INITIATED, OTHERWISE UNCOMMENT!!!!!
initialize_model_parallel(1)


> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


In [9]:
# Get Dataset.
from datasets import load_dataset
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]["text"]

# SlimPajama dataset
# https://huggingface.co/datasets/cerebras/SlimPajama-627B?row=0
# Just for now, we use a small portion of the dataset. Later, we can use more.
# data = load_dataset("cerebras/SlimPajama-627B", split="train").select(range(1000))
data = load_dataset("DKYoon/SlimPajama-6B", split="train").select(range(5000))

dataset = CustomDataset(data)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True)


Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

In [10]:
# Model Details
# model = Transformer(model_args)
# model = torch.nn.DataParallel(Transformer(model_args))
model = torch.nn.parallel.DistributedDataParallel(Transformer(model_args))
criterion = torch.nn.CrossEntropyLoss()


In [11]:
## Re-initialize the learnable weights
import math
for name, param in model.named_parameters():
    if param.requires_grad:
        if param.dim() >= 2:
            torch.nn.init.kaiming_uniform_(param)
        else:
            # bias terms or 1D params, uniform init
            bound = 1 / math.sqrt(param.size(0))
            torch.nn.init.uniform_(param, -bound, bound)


In [12]:
# Move model to cuda
model = model.to(device)


In [13]:
# Declare optimizer with finalized model parameters (after parallel + initialization)
optimizer = torch.optim.AdamW(model.parameters(), lr=3.0E-4)


In [14]:
# See how large the model is.
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")

trainable_params = sum(
	p.numel() for p in model.parameters() if p.requires_grad
)
print(f"Trainable number of parameters: {trainable_params}")


Total number of parameters: 166740992
Trainable number of parameters: 166740992


In [15]:
# Storage of loss metrics
import pickle
def write_to_file(epoch_number, loss_values, avg_loss, file_path):    
    # Open the file in binary append mode
    with open(file_path, 'ab') as file:
        # Serialize and write the epoch number
        pickle.dump(f'Epoch {epoch_number}', file)
        # Serialize and write the list of loss values
        pickle.dump(loss_values, file)
        # Average loss for the epoch
        pickle.dump(avg_loss, file)


In [16]:
# raise Exception("Please do not overwrite the files to avoid data losss; comment this line once done.")
SAVE_PATH = "./checkpoints/baseline_slimpj_big_sample.pt"
LOSSES_PATH = "./logs/losses/baseline_losses_slimpj_big_sample.txt"
TIMES_PATH = "./logs/times/baseline_slimpj_big_sample_elapsed_time.txt"
EPOCHS = 1


In [17]:
# Check the storage of the model
checkpoint = {
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    "epoch": -1,
    "loss": -1,
}
torch.save(checkpoint, SAVE_PATH)
del checkpoint


In [18]:
# Check logging of losses
write_to_file(-1, [-1, -2, -3], [-2], LOSSES_PATH)


In [19]:
import time


# Training

In [20]:
losses = []
# Log a 100K losses
log_every = max(1, (EPOCHS * len(data_loader)) // 100_000)
print("Number of batches:", len(data_loader))
model.train()
start_time = time.time()
# with torch.autograd.detect_anomaly():
for epoch in range(EPOCHS):
    total_loss = 0
    for c, batch in tqdm(enumerate(data_loader)):
        # Batch size x Max Seq LEn
        sample = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=model_args.max_seq_len)["input_ids"].to(device)
        target = sample.detach()

        # Batch size x Max Seq Len x Vocab Size
        optimizer.zero_grad()
        prediction = model(sample, 0)

        # Ensure swapping of axes
        loss = criterion(prediction.transpose(1, 2), target)
        loss.backward()

        # Loss logging
        total_loss += loss.item()
        if c % log_every == 0:
            print(f"Step: {c}, Loss: {loss.item():.4f}")
            losses.append(loss.item())

        # Change model weights
        optimizer.step()

        # Explicit destruction (may not be needed after  previous debugging)
        del loss, prediction, sample, target

    print(f"Epoch {epoch} Complete")
    avg_loss = total_loss / len(data_loader)
    checkpoint = {
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "epoch": epoch,
        "loss": avg_loss,
    }
    torch.save(checkpoint, SAVE_PATH)
    write_to_file(epoch, losses, avg_loss, LOSSES_PATH)
    losses.clear()

end_time = time.time()


Number of batches: 156


1it [00:01,  1.60s/it]

Step: 0, Loss: 10.3782


2it [00:02,  1.28s/it]

Step: 1, Loss: 10.2651


...

Step: 153, Loss: 3.8651


155it [02:48,  1.10s/it]

Step: 154, Loss: 3.8129


156it [02:49,  1.09s/it]

Step: 155, Loss: 4.0546
Epoch 0 Complete


In [21]:
optimizer.zero_grad()


In [22]:
elapsed_time = end_time - start_time
with open(TIMES_PATH, "a") as file:
    file.write(f"{elapsed_time}")


In [23]:
elapsed_time


184.83383345603943

In [24]:
cleanup()


Bad pipe message: %s [b'\xfcp\x1dG\xc1\xb3\xda\x93I\xaeK\xcf\xbe\xfa\x1fid\x92 \xf7=/\x82}\xb5\x00F\x7f\x87\xec\xa6H)/\xf3c\x82\x95%2\xd6X\x88\xaf\xe1\xafy\xb1\r\xe4\xb5\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \xb4\xe8-\x1a\x97f^b\xd4@H\xac\xc9\xbd \xb0_\xe3\xaf\x8e\xec\xc7\xc1\xe6']
Bad pipe message: %s [b'oL*\xf1T"\xa9\xb4\xba\xd3|\xaf\xaajn&r\x87\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0\'\x00g\x00@\xc0\n\xc0\x14\x009

With the following configuration:

@dataclass
class ModelArgs:
    dim: int = 1024
    n_layers: int = 8
    n_heads: int = 8
    n_kv_heads: Optional[int] = None
    vocab_size: int = -1  # defined later by tokenizer
    multiple_of: int = 64  # make SwiGLU hidden layer size multiple of large power of 2
    ffn_dim_multiplier: Optional[float] = None
    norm_eps: float = 1e-5

    max_batch_size: int = 32
    max_seq_len: int = 512

There are 166740992 parameters

To train across 156 batches -> 4992 Rows (32 sized batch), it took 185 sec

The model trains at 1619 Rows per Minute. -> Round down to 1500 Rows per minute.

1440 minutes per day.

With 1.5M Rows, we train for 

1.5M * 1 / 1500 * 1 / 1440 = 0.7 days